<a href="https://colab.research.google.com/github/MaJu502/lab5_paralela/blob/main/lab5_paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio no. 5 Programación Paralela
## author: Marco Jurado 20308

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-fwl6bzde
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-fwl6bzde
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=77f13169f2ebb7fc1fa79719b00acd3f80eadc2b686b22602cd01d18f8407fcc
  Stored in directory: /tmp/pip-ephem-wheel-cache-91fkr8s0/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661265 sha256=1eed8d6dbed61c4e06d2c35e291e7454ac29eae4a6d095325e80ac83af4cccc6
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
Successfully built pycuda


In [5]:
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


Ahora que el ambiente esta listo para ser ejecutado con CUDA procedemos a realizar los ejercicios de esta hoja de trabajo.

In [25]:
%%cuda --name lab5.cu

#include <stdio.h>
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda_runtime.h>


__global__ void vectorAdd(float *a, float *b, float *c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
        printf("Thread no. %d: %.2f + %.2f = %.2f\n", idx, a[idx], b[idx], c[idx]);
    }
}

int main() {
    int n = 500;  // Tamaño de los vectores
    float *h_a, *h_b, *h_c; // Vectores en la CPU
    float *d_a, *d_b, *d_c; // Vectores en la GPU
    size_t size = n * sizeof(float);

    // Alojar memoria en la CPU
    h_a = (float *)malloc(size);
    h_b = (float *)malloc(size);
    h_c = (float *)malloc(size);

    if (h_a == nullptr || h_b == nullptr || h_c == nullptr) {
        std::cerr << "Error al alojar memoria en la CPU." << std::endl;
        return 1;
    }

    // Inicializar los vectores en la CPU con valores aleatorios
    srand(time(NULL));
    for (int i = 0; i < n; i++) {
        h_a[i] = static_cast<float>(rand()) / RAND_MAX;
        h_b[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Alojar memoria en la GPU
    cudaError_t cudaStatus;
    cudaStatus = cudaMalloc(&d_a, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_b, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMalloc(&d_c, size);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al alojar memoria en la GPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar datos desde la CPU a la GPU
    cudaStatus = cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_a: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }
    cudaStatus = cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la CPU a la GPU para d_b: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Configuración de la cuadrícula y bloque
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Lanzar el kernel de CUDA
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    cudaStatus = cudaGetLastError();
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al lanzar el kernel de CUDA: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Copiar el resultado de la GPU a la CPU
    cudaStatus = cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    if (cudaStatus != cudaSuccess) {
        std::cerr << "Error al copiar datos desde la GPU a la CPU para d_c: " << cudaGetErrorString(cudaStatus) << std::endl;
        return 1;
    }

    // Imprimir el vector resultante y la suma
    float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        std::cout << "Resultado[" << i << "]: " << h_c[i] << std::endl;
        sum += h_c[i];
    }
    std::cout << "Suma total: " << sum << std::endl;

    // Liberar memoria
    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

'File written in /content/src/lab5.cu'

In [26]:
!nvcc -arch=sm_75 /content/src/lab5.cu -o "/content/src/lab5.o"

In [27]:
!chmod 755 /content/src/lab5.o
!/content/src/lab5.o

Thread no. 480: 0.68 + 0.01 = 0.69
Thread no. 481: 0.49 + 0.02 = 0.51
Thread no. 482: 0.33 + 0.38 = 0.70
Thread no. 483: 0.48 + 0.89 = 1.37
Thread no. 484: 0.74 + 0.02 = 0.76
Thread no. 485: 0.80 + 0.51 = 1.31
Thread no. 486: 0.73 + 0.28 = 1.00
Thread no. 487: 0.38 + 0.56 = 0.95
Thread no. 488: 0.20 + 0.07 = 0.27
Thread no. 489: 0.11 + 0.61 = 0.72
Thread no. 490: 0.97 + 0.23 = 1.20
Thread no. 491: 0.80 + 0.89 = 1.69
Thread no. 492: 0.78 + 0.80 = 1.58
Thread no. 493: 0.87 + 0.40 = 1.27
Thread no. 494: 0.64 + 0.92 = 1.56
Thread no. 495: 0.05 + 0.32 = 0.37
Thread no. 496: 0.93 + 0.54 = 1.47
Thread no. 497: 0.34 + 0.25 = 0.59
Thread no. 498: 0.92 + 0.81 = 1.73
Thread no. 499: 0.14 + 0.66 = 0.80
Thread no. 448: 1.00 + 0.98 = 1.98
Thread no. 449: 0.99 + 0.91 = 1.89
Thread no. 450: 0.79 + 0.51 = 1.30
Thread no. 451: 0.34 + 0.05 = 0.40
Thread no. 452: 0.12 + 0.25 = 0.37
Thread no. 453: 0.48 + 0.08 = 0.55
Thread no. 454: 0.35 + 0.40 = 0.75
Thread no. 455: 0.17 + 0.56 = 0.74
Thread no. 456: 0.11